# Actividad práctica: Predicción de calidad de aire

Descargue e importe el siguiente dataset usando pandas

    https://archive.ics.uci.edu/ml/datasets/Air+Quality

El objetivo de esta actividad es entrenar un modelo para predecir 
- la concentración de Monóxido de Carbono (CO)
- la concentración de Benzeno (C6H6)

> Observe como se comportan los datos, en base a esto proponga un modelo lineal en sus parámetros

Para entrenar su predictor use
- los primeros dos meses de datos para ajustar su predictor
- el tercer mes de datos para validar sus parámetros 


Nota: Un valor de "-200" corresponde a un *missing value*

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Importar datos

In [ ]:
!wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip
!unzip AirQualityUCI.zip

df = pd.read_csv("AirQualityUCI.csv", sep=';', decimal=',', na_values=[-200])
df = df[df.columns[:-2]] 
df["Date_time"] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format="%d/%m/%Y %H.%M.%S")
df.drop(["Date", "Time"], axis=1, inplace=True)
df.head(n=5)
df.set_index("Date_time", inplace=True)

!rm AirQualityUCI*

Visualizar primeros dos meses para la variable CO

Existe un comportamiento periódico diario y semanal

Un modelo sinusoidal podría ser apropiado

In [ ]:
sub_df = df.loc[:pd.to_datetime("06/10/2004 18:00:00")][["CO(GT)", "T", "RH", "AH"]]
sub_df.dropna(inplace=True)
time, data = sub_df.index, sub_df.values

fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
ax.plot(time, data[:, 0])
fig.autofmt_xdate()

# Entrenamiento en primeros dos meses

Entrenemos nuestro regresor lineal en sus parámetros

- Usaremos tiempo en "días" y valores normalizados para todas las variables
- Usaremos el valor anterior y el ante-anterior de CO
- Usaremos una regresión trigonométrica
- Utilizaremos una media flotante (sesgo)
- Usaremos la presión y la temperatura

In [ ]:
data_float = data.astype('float32')
data_float = (data_float - np.mean(data_float, axis=0, keepdims=True))/np.std(data_float, axis=0, keepdims=True)
time_float = np.array([t.timestamp()/(24*3600) - time[0].timestamp()/(24*3600) for t in time])

freqs = [*range(1, 10), 1/7, 1/14, 1/28]
lags = [*range(1, 10)]
X = np.hstack([np.ones(shape=(len(data_float), 1)), # sesgo
               data_float[:, 1].reshape(-1, 1), # Temperatura
               data_float[:, 2].reshape(-1, 1), # Presión relativa                      
               #np.stack([np.roll(time_float, lag) for lag in lags]).T, # Valores pasados
               np.stack([np.roll(data_float[:, 0], lag) for lag in lags]).T, # Valores pasados
               np.stack([np.cos(2*np.pi*time_float*f) for f in freqs]).T,  # Diccionario coseno
               np.stack([np.sin(2*np.pi*time_float*f) for f in freqs]).T, # Diccionario seno
               ])
Y = data_float[:, 0]

MSE = lambda real, estimation : np.mean(np.power(real - estimation, 2))

mask = (time_float <= 60) & (time_float > 2) # Primer mes
theta, _, _, _ = np.linalg.lstsq(X[mask, :], Y[mask], rcond=None)
display(theta)
display(f"Error cuadrático medio {MSE(Y[mask], np.dot(X[mask, :], theta))}")

fig, ax = plt.subplots(2, 1, figsize=(6, 4), tight_layout=True)
ax[0].plot(time_float[mask], data_float[mask, 0], label='data')
ax[0].plot(time_float[mask], np.dot(X[mask, :], theta), label='modelo')
ax[0].legend()
ax[1].plot(time_float[mask], np.abs(Y[mask] - np.dot(X[mask, :], theta)))
ax[1].set_ylabel('Error absoluto');

# Evaluación en mes final

In [ ]:
mask = time_float > 60
display(f"Error cuadrático medio {MSE(Y[mask], np.dot(X[mask, :], theta))}")

fig, ax = plt.subplots(1, figsize=(6, 3), tight_layout=True)
ax.plot(time_float[mask], data_float[mask, 0])
ax.plot(time_float[mask], np.dot(X[mask, :], theta));

Tiene nuestra solución menor MSE que simplemente predecir con el valor anterior?

In [ ]:
MSE(Y[mask], Y[np.where(mask)[0]-1])

Tiene nuestra solución menor MSE que simplemente predecir con el valor medio?

In [ ]:
MSE(Y[mask], np.mean(Y))